# Raw data
--------------
## first check the raw data

In [4]:
# check how many lines
! wc -l data/emojitweets-01-04-2018.txt

18866900 data/emojitweets-01-04-2018.txt


In [2]:
# check how many words
! wc -w data/emojitweets-01-04-2018.txt

209430000 data/emojitweets-01-04-2018.txt


In [3]:
# On average the sentense are have 11 words
209430000/18866900

11.100392751326398

## Train - test - validation split

In [6]:
# split on 80-10-10
18866900 * 0.8, 18866900 * 0.1, 18866900 * 0.2

(15093520.0, 1886690.0, 3773380.0)

In [8]:
! head -15093520 data/emojitweets-01-04-2018.txt >>emoji_train.txt


In [14]:
! (tail -3773380 data/emojitweets-01-04-2018.txt | head -1886690) >> emoji_val.txt

tail: error writing 'standard output': Broken pipe


In [10]:
! tail -1886690 data/emojitweets-01-04-2018.txt >> emoji_test.txt

In [1]:
# generate small_sample
! head -1509352 data/emoji_train.txt >> data/emoji_ss.txt

# Word Vectorize

In [26]:
# Python program to generate word vectors using Word2Vec 
  
# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec

import emoji
import regex

import pandas as pd
import numpy as np
import pickle


In [6]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer


In [72]:
# sample = open("emoji_1M.txt", "r") 
sample = open("emoji_train.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
f = s.replace("\n", " ") 



# replace all 
signs = [",", ".", "(" ,")", "'", '/"', ":", ";", "/", "(", ")", "“", "”", "-", "+", "#"]
for sign in signs:
    f = f.replace(sign, "")
  
data = [] 
  

In [8]:
# move to helper
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

### Small Sample

In [4]:
# sample = open("../emoji_100k.txt", "r") 
# s_ss = sample.read() 
  
# # Replaces escape character with space
# f_ss = s_ss.lower().replace("\n", " ")

# # replace all 
# signs = [",", ".", "(" ,")", "'", '/"', ":", ";", "/", "(", ")"]
# for sign in signs:
#     f_ss = f_ss.replace(sign, "")
  
# data = [] 

In [73]:
# f = f_ss
# iterate through each sentence in the file 
for i in sent_tokenize(f): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

In [74]:
# import stopwords
stop = set(stopwords.words('english'))



In [ ]:
# remove stop words and snowball stemmer
out = [[word for word in words if word not in stop] for words in data]
snowball = SnowballStemmer('english')
docs_snowball = [[snowball.stem(word) for word in words] for words in out]

In [12]:
docs_snowball[0]

['squad',
 'arriv',
 'game',
 '2',
 '🚀',
 'dude',
 'like',
 '5',
 '’',
 '8',
 '140',
 'pound',
 'dick',
 'long',
 'strongalway',
 'littl',
 'dude',
 'carri',
 '🍆',
 '\U0001f92a🙃',
 'followers👇',
 'cant',
 'breatiuhw',
 '💀💀💀',
 '2️⃣4️⃣',
 'hour',
 'til',
 'schedul',
 'drop',
 '!']

save emoji list

In [10]:
# emoji_list = list(emoji_set)
# with open("emoji_list.csv", 'w', newline='') as myfile:
#      w = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#      w.writerow(emoji_list)

4358

In [ ]:
with open("emoji_list.csv", 'r', newline='') as myfile:
     emoji_list = csv.reader(myfile, quoting=csv.QUOTE_ALL)

# CBOW model

In [13]:
# Create CBOW model 
model1 = gensim.models.Word2Vec(docs_snowball, min_count = 10,  
                              size = 200, window = 5,
                                workers=8 ) 

In [15]:
# Save CBOW model
import pickle
with open('cbow_ss.pkl', 'wb') as f:
    pickle.dump(model1, f)

In [14]:
model1.estimate_memory()

{'vocab': 19961500,
 'vectors': 31938400,
 'syn1neg': 31938400,
 'total': 83838300}

In [15]:
val_df = pd.read_csv("data/emoji_val.2csv",index_col=0)
emoji_list = val_df["emoji"].values.tolist()

In [41]:
# move to helper
def emoji_prodictor(word, model):
    sim = model.most_similar(word,topn=75)
    for i in sim:
        if i[0] in emoji_list:
            return i[0]
    return word

In [19]:
# move to helper
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out
    
    
s1 = "let's go grocery and buy some apple"
word_pipeline(s1)

['let', 'go', 'groceri', 'buy', 'appl']

In [18]:
s1 = "let's go grocery and buy some apple"
for w in word_pipeline(s1):
    print(emoji_prodictor(w, model1))


🙅🏽‍♀️
😴
🛍
buy
🎹


In [20]:
emoji_prodictor("today")

'👌🏼'

# skip-gram model

In [21]:
# Create skip-gram model 
model2 = gensim.models.Word2Vec(docs_snowball, 
                                min_count = 5,  
                                size = 100, 
                                window = 5,
                                sg = 1,
                                workers=8) 

In [41]:
# Save skip-gram model
import pickle
with open('sg_ss.pkl', 'wb') as f:
    pickle.dump(model2, f)

In [23]:
s1 = "let's go grocery and buy some apple"
for w in word_pipeline(s1):
    print(emoji_prodictor(w, model2))


let
go
groceri
buy
appl


In [58]:
model2.most_similar("am",topn=20)

[('lt——', 0.8018146753311157),
 ('recover…', 0.7987035512924194),
 ('wichita', 0.7942888140678406),
 ('hoeless', 0.7892517447471619),
 ('👎🏼👎🏼', 0.7889255285263062),
 ('philanthropi', 0.7855656147003174),
 ('concess', 0.783519446849823),
 ('earplug', 0.7825350165367126),
 ('£10k', 0.7814924716949463),
 ('conti', 0.7789944410324097),
 ('2k16', 0.7770678400993347),
 ('braai', 0.7757090926170349),
 ('👍⚽️', 0.7756443023681641),
 ('azpilicueta', 0.7747820615768433),
 ('☪️', 0.7731027007102966),
 ('👩\u200d💼', 0.7727632522583008),
 ('ravag', 0.772744357585907),
 ('person❤️', 0.7700779438018799),
 ('girls-', 0.7693312168121338),
 ('🤔is', 0.7692635655403137)]

-----------------
validation set

In [52]:
val_df = pd.read_csv("data/emoji_val.2csv",index_col=0)
emoji_set = val_df["emoji"].values.tolist()

In [34]:
val_df["full_desc"] = val_df.full_desc.apply(lambda x: x.strip('][').split(', '))
val_df["desc2"] = val_df.desc2.apply(lambda x: x.strip('][').split(', ') if isinstance(x,str) else "")

In [36]:
for i in val_df.index:
    for j in range(len(val_df.full_desc[i])):
#         print(i)
        val_df.full_desc[i][j] = val_df.full_desc[i][j].strip("/'")
    if val_df.desc2[i]:
        for k in range(len(val_df.desc2[i])):
    #         print(i)
            val_df.desc2[i][k] = val_df.desc2[i][k].strip("/'")

In [38]:
val_df["desc2"][0][1]

'grin'

In [40]:
# build validation set
val_dict = {}
lst = []
for i in val_df.index.tolist():
    lst.extend(val_df.full_desc[i])
    lst.extend(val_df.desc2[i])

In [41]:
vals = list(set(lst))
for val in vals:
    val_dict[val] = []
    for index in val_df.index:
        
        if val in val_df.full_desc[index]:
            val_dict[val].append(val_df.emoji[index])
        elif val in val_df.desc2[index]:
            val_dict[val].append(val_df.emoji[index])


In [ ]:


with open('val_dict.pickle', 'wb') as handle:
    pickle.dump(val_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
with open('val_dict.pickle', 'rb') as handle:
    val_dict = pickle.load(handle)

# print val_dict == b

In [53]:
def word2vec_score(val_dict, model):
    total_words = 0
    total_predicts = 0
    correct = 0
    words = []
    word_vectors = model.wv
    for word in val_dict:
#         print(word)
        if word_pipeline(word):
            w = word_pipeline(word)[0]
            words.append(w)
            if w in word_vectors:
                total_words += 1
                if emoji_prodictor(w, model) in val_dict[word]:
                    correct += 1
                    total_predicts += 1
                elif emoji_prodictor(w, model) in emoji_set:
                    total_predicts += 1
    return correct/total_words, correct/total_predicts, words
            

In [54]:
score1 = word2vec_score(val_dict, model1)

In [55]:
score2 = word2vec_score(val_dict, model2)

In [56]:
# score for CBOW and skipgram for small sample with SnowballStemmer
score1[0:2],score2[0:2]

((0.14964788732394366, 0.18695014662756598),
 (0.15307262569832403, 0.23539518900343642))

In [ ]:
# score for CBOW and skipgram for full train sample with SnowballStemmer
score1[0:2],score2[0:2]

In [60]:
score1[2][::20]

['',
 'stethoscop',
 'yoga',
 'bellhop',
 'new',
 'fax',
 'red',
 'decor',
 'manicur',
 'rais',
 'beaver',
 'ballet',
 'guyana',
 'health',
 'faro',
 'footprint',
 'suspens',
 'superhero:',
 'zambia',
 'punch',
 'pill',
 'ok',
 'industri',
 'ice',
 'cuba',
 'curl',
 'die',
 'pancak',
 'mantilla',
 'milk',
 'skunk',
 'shop',
 'clean',
 'motorway',
 'backhand',
 'netherland',
 'quarter',
 'bermuda',
 'sew',
 'outlin',
 'wale',
 'hook',
 'state',
 'peopl',
 'polic',
 'island',
 'aquarius',
 'potabl',
 'cook',
 "o'clock",
 'gua',
 'two-hump',
 'nevi',
 'monkey',
 'christma',
 'young',
 'santa',
 'butterfli',
 'revolv',
 'mate',
 'emblem',
 'antarctica',
 'guardsman',
 'wrestl',
 'ferri',
 'cat',
 'england',
 'blowfish',
 'cross',
 'person',
 'luxembourg',
 'open',
 'comput',
 'unspecifi',
 'tab',
 'away',
 'eight-spok',
 'babi',
 'blond',
 'curri',
 'floppi',
 'ribbon',
 'samoa',
 'knife',
 'fantasi',
 'barthélemi',
 'race',
 'femal',
 'man',
 'cream',
 'stone',
 'bulgaria',
 'boat:',
 'co

In [62]:
model2.most_similar('money',topn=50)

[('📩💵', 0.7020508050918579),
 ('💰💯', 0.6812326312065125),
 ('🤷🏻\u200d♀️💰', 0.6755300760269165),
 ('🔥get', 0.6755296587944031),
 ('🤑💵', 0.6684743165969849),
 ('cash', 0.6657474040985107),
 ('ambani😂😂', 0.6505308747291565),
 ('tireddddd', 0.644366443157196),
 ('launder', 0.6404151320457458),
 ('🤑💰', 0.6393123269081116),
 ('rake', 0.6330356001853943),
 ('splurg', 0.6325438022613525),
 ('stack', 0.6323885321617126),
 ('🤦🏾\u200d♂️🤣🤣🤣🤣🤣', 0.6310147643089294),
 ('flow✨', 0.628703773021698),
 ('semperfi', 0.628624439239502),
 ('💳💵', 0.6212002038955688),
 ('🤣🤷🏾\u200d♀️', 0.620684802532196),
 ('taxpay', 0.6157459020614624),
 ('fiat', 0.615238606929779),
 ('💵💵💵', 0.6151927709579468),
 ('becu', 0.6137044429779053),
 ('heist', 0.6131662130355835),
 ('🇺🇸-armi', 0.612061083316803),
 ('busted👊', 0.611751139163971),
 ('basi', 0.610927939414978),
 ('shit😒💯', 0.608547568321228),
 ('🤦🏾\u200d♀️🙄', 0.6056149005889893),
 ('armani', 0.6028887629508972),
 ('💁🏼\u200d♀️😂', 0.6028388738632202),
 ('fuck😭', 0.60243

In [ ]:
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out

In [115]:
sentense = "under"
[word for word in sentense.lower().split(" ") if word not in stop]

[]

In [116]:
word_pipeline(sentense)model2

[]

In [65]:
emoji_prodictor('us', model1)

'😀'

In [71]:
emoji_prodictor('nyc', model1)

'🚌'

-------------------------
RNN model

In [ ]:
with open('sherlock_homes.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))
chars = sorted(list(set(text))) # getting all unique chars
print('total chars: ', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2)